# 任务二 - 特征工程
* 特征工程是在训练集上做 还是所有原始数据
* 这里选择在训练集上做特征工程

## 1. 特征选择

In [2]:
import numpy as np
import pandas as pd

In [51]:
train_data = pd.read_csv('./data/train.csv')
train_data.head()

,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,transd_mcc,trans_days_interval_filter,trans_days_interval,regional_mobility,...,loans_max_limit,loans_avg_limit,consfin_credit_limit,consfin_credibility,consfin_org_count_current,consfin_product_count,consfin_max_limit,consfin_avg_limit,latest_query_day,loans_latest_day
0,0.01,0.99,4000,0.96,1.00,0.405,16.0,29.0,28.0,1.0,...,5000.0,1845.0,10600.0,81.0,7.0,7.0,15600.0,8228.0,0.0,9.0
1,0.03,0.97,500,0.87,1.00,0.205,18.0,27.0,27.0,3.0,...,2600.0,1800.0,16300.0,78.0,5.0,5.0,21600.0,7160.0,30.0,27.0
2,0.02,0.86,0,1.98,0.70,0.205,18.0,53.0,33.0,2.0,...,2300.0,1500.0,10400.0,82.0,5.0,5.0,13800.0,10320.0,3.0,137.0
3,0.02,0.81,1000,1.49,0.73,0.555,23.0,15.0,8.0,4.0,...,4500.0,2500.0,6600.0,78.0,11.0,12.0,17400.0,6418.0,20.0,51.0
4,0.02,0.94,2000,1.28,1.00,0.458,19.0,30.0,14.0,4.0,...,3500.0,1758.0,15100.0,80.0,5.0,6.0,22800.0,9360.0,4.0,2.0


### 相关系数

In [61]:
#查看预测特征与其它特征之间的相关系数
corr = train_data.corr()["status"].sort_values(ascending = False)
corr[:20]

status                                     1.000000
trans_fail_top_count_enum_last_1_month     0.340668
history_fail_fee                           0.324228
loans_overdue_count                        0.289973
latest_one_month_fail                      0.249941
trans_fail_top_count_enum_last_6_month     0.141126
rank_trad_1_month                          0.139368
trans_fail_top_count_enum_last_12_month    0.121768
top_trans_count_last_1_month               0.112420
trans_day_last_12_month                    0.091685
latest_six_month_loan                      0.072033
latest_three_month_loan                    0.070949
latest_query_day                           0.069994
query_cash_count                           0.066914
avg_price_top_last_12_valid_month          0.066507
loans_count                                0.059762
low_volume_percent                         0.053561
consfin_product_count                      0.039973
consfin_org_count_current                  0.038730
consfin_org_

### 2.1 IV筛选特征
参考：https://www.jianshu.com/p/9a34892ee9e5

In [35]:
from sklearn.utils.multiclass import type_of_target
from scipy import stats
import math

#求woe值和iv值
def woe(X, y, event):
    res_woe = []  #列表存放woe字典
    res_iv = []  #列表存放iv
    X1 = feature_discretion(X) #对连续型特征进行处理
    for i in range(0, X1.shape[-1]):  #遍历所有特征
        x = X1[:, i]  #单个特征
        woe_dict, iv1 = woe_single_x(x, y, event)  #计算单个特征的woe值
        res_woe.append(woe_dict)
        res_iv.append(iv1)
    return np.array(res_woe), np.array(res_iv)  #返回数组

In [36]:
#求单个特征的woe值
def woe_single_x(x, y, event):
    event_total, non_event_total = count_binary(y, event) #计算好人坏人总数
    x_labels = np.unique(x) #特征中的分段
    woe_dict = {}  #存放每个分段的名称 以及 其对应的woe值
    iv = 0
    for x1 in x_labels: #遍历每个分段
        y1 = y[np.where(x == x1)[0]]
        event_count, non_event_count = count_binary(y1, event=event)
        rate_event = 1.0 * event_count / event_total
        rate_non_event = 1.0 * non_event_count / non_event_total
        #woe无穷大时处理
        if rate_event == 0:
            print()#print("{'",x1,"'}"+":全是好人") #只输出不做处理
        elif rate_non_event == 0:
            print()#print("{'",x1,"'}"+":全是坏人")
        else:
            woe1 = math.log(rate_event / rate_non_event)
            woe_dict[x1] = woe1  
            iv += (rate_event - rate_non_event) * woe1
    return woe_dict, iv

In [37]:
#计算个数
def count_binary(a, event):
    event_count = (a == event).sum()
    non_event_count = a.shape[-1] - event_count
    return event_count, non_event_count

In [38]:
#判断特征数据是否为离散型
def feature_discretion(X):
    temp = []
    for i in range(0, X.shape[-1]):
        x = X[:, i]
        x_type = type_of_target(x)
        if pd.Series(list(x)).dtype != 'O':
            x1 = discrete(x)
            temp.append(x1)
        else:
            temp.append(x)
    return np.array(temp).T

### 目标变量和特征特征

In [71]:
#目标变量
y = train_data['status'].values
# 所有特征
x = train_data.drop(['status'],axis=1).values
y

array([1, 0, 0, ..., 1, 0, 0], dtype=int64)

### 计算所有特征的iv

In [90]:
res_woe, res_iv = woe(x, y, y[0])
dic = dict(zip(train_data.columns.values, res_iv))  #特征列表:IV值列表
dic_sort = sorted(dic.items(), key=lambda x: x[1], reverse=True)
fout = open('./data/feature_iv.txt', 'w')
for feature,iv in dic_sort:
    fout.write('%s;%s\n'%(feature,iv))
fout.close()
dic_sort

[('trans_fail_top_count_enum_last_1_month', 0.5738251555141427),
 ('history_suc_fee', 0.5032245133297788),
 ('latest_six_month_apply', 0.43866404297963046),
 ('loans_settle_count', 0.35919819707466094),
 ('trans_day_last_12_month', 0.34408240900678505),
 ('trans_fail_top_count_enum_last_12_month', 0.2644876668331204),
 ('trans_fail_top_count_enum_last_6_month', 0.25640227596356635),
 ('latest_one_month_suc', 0.14714140965160658),
 ('history_fail_fee', 0.12851330530009836),
 ('rank_trad_1_month', 0.09658511076715653),
 ('max_cumulative_consume_later_1_month', 0.09410939090142396),
 ('first_transaction_day', 0.0874862341754116),
 ('trans_top_time_last_1_month', 0.0826299051197211),
 ('pawns_auctions_trusts_consume_last_1_month', 0.05951063024820402),
 ('top_trans_count_last_1_month', 0.05823744436103096),
 ('consfin_max_limit', 0.05796154598230204),
 ('trans_amount_3_month', 0.04338806330064936),
 ('loans_avg_limit', 0.04301830985857236),
 ('latest_one_month_loan', 0.03975843902475576),


### 2.2 随机森林
参考：https://github.com/jameszhenyu/ML_data/blob/master/feature_select.ipynb

In [73]:
from sklearn.ensemble import RandomForestRegressor

In [75]:
#目标变量 dataframe
y_train = train_data['status']
# 所有特征
x_train = train_data.drop(['status'],axis=1)

rf = RandomForestRegressor()
rf.fit(x_train, y_train)# x是所有特征  y是目标值
random_forrest = sorted(zip(map(lambda x: '%.4f'%x, rf.feature_importances_), list(x_train.columns)), reverse=True)

len(random_forrest)
random_forrest[:10]

D:\Program Files\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


[('0.0923', 'trans_fail_top_count_enum_last_1_month'),
 ('0.0881', 'history_fail_fee'),
 ('0.0332', 'loans_score'),
 ('0.0303', 'trans_day_last_12_month'),
 ('0.0234', 'trans_amount_3_month'),
 ('0.0219', 'repayment_capability'),
 ('0.0211', 'trans_amount_increase_rate_lately'),
 ('0.0204', 'latest_one_month_fail'),
 ('0.0201', 'apply_score'),
 ('0.0198', 'loans_latest_day')]